# Préparation et nettoyage des données DVF

**Objectif** : préparer un jeu de données propre pour l'analyse des **locaux commerciaux / boutiques** à **Paris** (arrondissements, prix/m², année, géolocalisation).



In [1]:
import pandas as pd
import numpy as np
import os

#### Lecture du fichier dvf.csv

On définit en premier lieu les **colonnes utiles** qui vont nous servir pour l'extraction et le nettoyage des données, dans le but d'**optimiser la lecture du fichier csv** qui est assez lourd (*+3Go*). Toujours dans le même objectif, la lecture se fait par **chunk** de 100000 lignes. Durant le processus, on réalise :
- La suppression des valeurs NaN
- La modification du type de la colonne 'date_mutation' en datetime
- La reduction des données au type "Local industriel, commercial ou assimilé"
- La filtration pour conserver uniquement les biens sur Paris

In [29]:
# Define columns to read upfront
colonnes_a_lire = [
    'date_mutation',
    'valeur_fonciere',
    'code_postal',
    'type_local',
    'surface_reelle_bati',
    'nombre_pieces_principales',
    'longitude',
    'latitude',
    'adresse_nom_voie',
    'nom_commune'
]

# Process CSV in chunks
chunk_size = 100000
filtered_chunks = []

print("Processing DVF file in chunks...")
for i, chunk in enumerate(pd.read_csv('dvf.csv', 
                                       usecols=colonnes_a_lire,
                                       chunksize=chunk_size,
                                       low_memory=False)):
    
    # Drop NaN values and create explicit copy
    chunk = chunk.dropna(subset=['valeur_fonciere', 'surface_reelle_bati', 
                                  'nombre_pieces_principales',
                                  'latitude', 'longitude']).copy()
    
    # Convert date
    chunk['date_mutation'] = pd.to_datetime(chunk['date_mutation'], errors='coerce')
    
    # Filter by type_local
    chunk = chunk[chunk['type_local'] == 'Local industriel. commercial ou assimilé'].copy()
    
    # Filter Paris postal codes
    chunk = chunk[chunk['code_postal'].astype(str).str.startswith('75')].copy()
    
    if len(chunk) > 0:
        filtered_chunks.append(chunk)
    
    print(f"Chunk {i+1}: retained {len(chunk)} rows")

# Combine all filtered chunks
df = pd.concat(filtered_chunks, ignore_index=True)
print(f"\nTotal rows after filtering: {len(df)}")

Processing DVF file in chunks...
Chunk 1: retained 0 rows
Chunk 2: retained 6 rows
Chunk 3: retained 19 rows
Chunk 4: retained 0 rows
Chunk 5: retained 0 rows
Chunk 6: retained 0 rows
Chunk 7: retained 0 rows
Chunk 8: retained 0 rows
Chunk 9: retained 0 rows
Chunk 10: retained 0 rows
Chunk 11: retained 0 rows
Chunk 12: retained 0 rows
Chunk 13: retained 0 rows
Chunk 14: retained 0 rows
Chunk 15: retained 0 rows
Chunk 16: retained 0 rows
Chunk 17: retained 0 rows
Chunk 18: retained 0 rows
Chunk 19: retained 0 rows
Chunk 20: retained 0 rows
Chunk 21: retained 0 rows
Chunk 22: retained 0 rows
Chunk 23: retained 0 rows
Chunk 24: retained 0 rows
Chunk 25: retained 0 rows
Chunk 26: retained 0 rows
Chunk 27: retained 0 rows
Chunk 28: retained 0 rows
Chunk 29: retained 0 rows
Chunk 30: retained 0 rows
Chunk 31: retained 0 rows
Chunk 32: retained 0 rows
Chunk 33: retained 1 rows
Chunk 34: retained 0 rows
Chunk 35: retained 3955 rows
Chunk 36: retained 1782 rows
Chunk 37: retained 0 rows
Chunk 3

##### On vérifie si les colonnes restantes contiennent des valeurs NaN

In [30]:
df.isna().sum()

date_mutation                0
valeur_fonciere              0
adresse_nom_voie             0
code_postal                  0
nom_commune                  0
type_local                   0
surface_reelle_bati          0
nombre_pieces_principales    0
longitude                    0
latitude                     0
dtype: int64

##### Puis on retire du dataframe la colonne "type_local"

In [31]:
df = df.drop('type_local', axis=1)

##### Création d'une feature prix m²

In [32]:
df['prix_m2'] = df['valeur_fonciere'] / df['surface_reelle_bati']

##### On supprime les valeurs aberrantes au m²

In [33]:
df = df[(df['prix_m2'] > 500) & (df['prix_m2'] < 50000)]

##### Extraction de l'arrondissement (normalisation de code_postal)

In [34]:
df['code_postal'] = df['code_postal'].astype(int).astype(str).str.zfill(5)
df['arrondissement'] = pd.to_numeric(df['code_postal'].str[-2:], errors='coerce')
df.loc[~df['arrondissement'].between(1,20), 'arrondissement'] = np.nan
df['arrondissement'] = df['arrondissement'].dropna().astype(int)

##### On supprime la colonne code_postal (c'est toujours Paris)

In [35]:
df = df.drop('code_postal', axis=1)

##### Certains noms de communes ne sont pas Paris, il faut donc les supprimer

In [36]:
df = df[df['nom_commune'].str.startswith('Paris')]
df.shape

(17586, 10)

# Nettoyage BDD Loyer

In [37]:
loyer_df = pd.read_csv("loyers.csv", sep=';', encoding='utf-8')

##### On supprime les colonnes inutiles

In [38]:
colonnes_a_supprimer = ["Secteurs géographiques", "Numéro du quartier", "Ville","geo_shape"]
loyer_df = loyer_df.drop(columns=colonnes_a_supprimer, errors='ignore')

##### Le dataframe ne contient pas de valeurs NaN ou null, ni de doublons

##### Nettoyage des coordonnées GPS (format "lat, lon")

In [39]:
loyer_df[['lat', 'lon']] = loyer_df['geo_point_2d'].str.split(',', expand=True)
loyer_df['lat'] = loyer_df['lat'].astype(float)
loyer_df['lon'] = loyer_df['lon'].astype(float)

##### Suppression de la colonne 'geo_point_2d'

In [40]:
loyer_df = loyer_df.drop('geo_point_2d', axis=1)

In [41]:
df.head()

,date_mutation,valeur_fonciere,adresse_nom_voie,nom_commune,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,prix_m2,arrondissement
26,2020-01-06,878378.0,BD DE LA MADELEINE,Paris 1er Arrondissement,43.0,0.0,2.326882,48.869331,20427.395349,1.0
27,2020-01-13,475000.0,RUE DE WASHINGTON,Paris 8e Arrondissement,50.0,0.0,2.301658,48.872630,9500.000000,8.0
28,2020-01-03,800000.0,BD DE SEBASTOPOL,Paris 3e Arrondissement,125.0,0.0,2.351054,48.863055,6400.000000,3.0
29,2020-01-06,710000.0,RUE SAINT HONORE,Paris 1er Arrondissement,30.0,0.0,2.326348,48.866939,23666.666667,1.0
31,2020-01-09,17400000.0,RUE DU FAUBOURG SAINT HONORE,Paris 8e Arrondissement,387.0,0.0,2.309755,48.873496,44961.240310,8.0


# Jointure du dataframe des deux dataframes

In [42]:
from scipy.spatial import cKDTree

In [43]:
coords_quartiers = loyer_df[['lat', 'lon']].values
tree = cKDTree(coords_quartiers)

In [44]:
coords_dvf = df[['latitude', 'longitude']].values

In [45]:
distances, indices = tree.query(coords_dvf)

#### Ajouter les colonnes au DataFrame DVF

In [46]:
df['quartier'] = loyer_df.iloc[indices]['Nom du quartier'].values
df['numero_insee_quartier'] = loyer_df.iloc[indices]['Numéro INSEE du quartier'].values
df['loyer_ref_m2'] = loyer_df.iloc[indices]['Loyers de référence'].values
df['loyer_majore_m2'] = loyer_df.iloc[indices]['Loyers de référence majorés'].values
df['loyer_minore_m2'] = loyer_df.iloc[indices]['Loyers de référence minorés'].values
df['distance_quartier_m'] = distances * 111000  # Conversion approximative en mètres

#### Checking s'il y a pas des lignes ou le quartier est loin de la réalité

In [47]:
df = df[df['distance_quartier_m'] <= 1000]

In [48]:
df.head()

,date_mutation,valeur_fonciere,adresse_nom_voie,nom_commune,surface_reelle_bati,nombre_pieces_principales,longitude,latitude,prix_m2,arrondissement,quartier,numero_insee_quartier,loyer_ref_m2,loyer_majore_m2,loyer_minore_m2,distance_quartier_m
26,2020-01-06,878378.0,BD DE LA MADELEINE,Paris 1er Arrondissement,43.0,0.0,2.326882,48.869331,20427.395349,1.0,Place-Vendôme,7510104,29.3,35.16,20.51,318.554490
27,2020-01-13,475000.0,RUE DE WASHINGTON,Paris 8e Arrondissement,50.0,0.0,2.301658,48.872630,9500.000000,8.0,Faubourg-du-Roule,7510830,22.8,27.40,16.00,320.218020
28,2020-01-03,800000.0,BD DE SEBASTOPOL,Paris 3e Arrondissement,125.0,0.0,2.351054,48.863055,6400.000000,3.0,Sainte-Avoie,7510312,31.8,38.16,22.26,425.130070
29,2020-01-06,710000.0,RUE SAINT HONORE,Paris 1er Arrondissement,30.0,0.0,2.326348,48.866939,23666.666667,1.0,Place-Vendôme,7510104,29.3,35.16,20.51,248.094155
31,2020-01-09,17400000.0,RUE DU FAUBOURG SAINT HONORE,Paris 8e Arrondissement,387.0,0.0,2.309755,48.873496,44961.240310,8.0,Faubourg-du-Roule,7510830,22.8,27.40,16.00,629.632252


In [49]:
df['nombre_pieces_principales'].value_counts()

nombre_pieces_principales
0.0    16486
Name: count, dtype: int64

#### Création du fichier final dvf_loyers.csv

In [50]:
df.to_csv("dvf_loyers.csv", sep=";", encoding="utf-8", index=False)
df.shape

(16486, 16)